In [1]:
import os#环境代理设置
os.environ["http_proxy"] = "http://127.0.0.1:7890"
os.environ["https_proxy"] = "http://127.0.0.1:7890"

In [2]:
import torch
from transformers.models.bert import BertModel, BertTokenizer

In [13]:
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name, output_hidden_states=True)

In [14]:
model.config

BertConfig {
  "_attn_implementation_autoset": true,
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_hidden_states": true,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.46.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

In [15]:
model

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [16]:
test_sentence = 'this is a test sentence'
model_input = tokenizer(test_sentence, return_tensors='pt')

In [17]:
model.eval()
with torch.no_grad():
    output = model(**model_input)

In [18]:
output.keys()

odict_keys(['last_hidden_state', 'pooler_output', 'hidden_states'])

In [21]:
output[2][0].shape # embedding

torch.Size([1, 7, 768])

In [30]:
output[2][1].shape # first layer output

torch.Size([1, 7, 768])

### from scratch

- BertLayer
    - attention: BertAttention
        - self: BertSelfAttention
        - output: BertSelfOutput
    - intermediate: BertIntermediate
    - output: BertOutput

In [34]:
embedding = output[2][0]

In [36]:
layer = model.encoder.layer[0]

In [44]:
mha_output = layer.attention.self(embedding)

In [63]:
mha_output[0].shape

torch.Size([1, 7, 768])

In [64]:
attn_output = layer.attention.output(mha_output[0], embedding)

In [51]:
attn_output.shape

torch.Size([1, 7, 768])

In [52]:
layer

BertLayer(
  (attention): BertAttention(
    (self): BertSelfAttention(
      (query): Linear(in_features=768, out_features=768, bias=True)
      (key): Linear(in_features=768, out_features=768, bias=True)
      (value): Linear(in_features=768, out_features=768, bias=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (output): BertSelfOutput(
      (dense): Linear(in_features=768, out_features=768, bias=True)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
  )
  (intermediate): BertIntermediate(
    (dense): Linear(in_features=768, out_features=3072, bias=True)
    (intermediate_act_fn): GELUActivation()
  )
  (output): BertOutput(
    (dense): Linear(in_features=3072, out_features=768, bias=True)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
)

In [65]:
mlp1 = layer.intermediate(attn_output)

In [59]:
mlp1.shape

torch.Size([1, 7, 3072])

In [66]:
mlp2 = layer.output(mlp1, attn_output)

In [60]:
mlp2.shape

torch.Size([1, 7, 768])

In [ ]:
mlp2

tensor([[[ 0.1556, -0.0080, -0.0707,  ...,  0.0786,  0.0213,  0.0616],
         [-0.5333,  0.5799,  0.1044,  ...,  0.0241,  0.4888,  0.0161],
         [-1.0609, -0.3058, -0.5043,  ...,  0.1874,  0.2874,  0.4032],
         ...,
         [ 0.8206, -0.6656, -0.7054,  ...,  0.1347,  0.1117, -1.9040],
         [ 1.1128,  0.6603, -0.1509,  ...,  0.3253, -1.0006, -1.9106],
         [-0.0736,  0.0346,  0.0376,  ..., -0.4506,  0.6585, -0.0502]]],
       grad_fn=<NativeLayerNormBackward0>)

In [70]:
output[2][1]

tensor([[[ 0.1556, -0.0080, -0.0707,  ...,  0.0786,  0.0213,  0.0616],
         [-0.5333,  0.5799,  0.1044,  ...,  0.0241,  0.4888,  0.0161],
         [-1.0609, -0.3058, -0.5043,  ...,  0.1874,  0.2874,  0.4032],
         ...,
         [ 0.8206, -0.6656, -0.7054,  ...,  0.1347,  0.1117, -1.9040],
         [ 1.1128,  0.6603, -0.1509,  ...,  0.3253, -1.0006, -1.9106],
         [-0.0736,  0.0346,  0.0376,  ..., -0.4506,  0.6585, -0.0502]]])